优化算法:使用Keras提供的其他优化器，如梯度下降，看在其他算法下模型参数对模型训练和过拟合的速度有怎样的影响。
损失函数:尝试使用Keras其他可用的损失函数，探究选用其他的损失函数是否可以提升模型的性能。
学习率与迭代次数更新策略
更大的Batch Size:使用更大的Batch Size意味着模型在训练集和测试集上的数据操作规模更大了

In [1]:
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.applications.inception_v3 import preprocess_input
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from keras.layers import *
#from keras.layers import Input
from keras.models import *
#from keras.models import Model
#from keras.layers import Input, Dense
#a = Input(shape=(32,))
#b = Dense(32)(a)
#model = Model(inputs=a, outputs=b)
from keras.callbacks import *
from keras.optimizers import *
from keras.regularizers import *
from keras import initializers
from keras.applications import *

#from keras.utils import multi_gpu_model 
#from keras.applications import ResNet50
#from keras.applications import VGG16
#from keras.applications import VGG19
#from keras.applications import Xception # TensorFlow ONLY
#from keras.applications import InceptionResNetV2
#from keras.applications import InceptionV3

#tf.keras.applications.inception_v3.InceptionV3
#tf.keras.applications.inception_resnet_v2.InceptionResNetV2
####################################################################
#设置GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

#设置项
#看具体的模型参数设置在:https://www.tensorflow.org/api_docs/python/tf/keras/applications
MODELS = {"ResNet50":ResNet50}
#MODELS = {"NASNetLarge":NASNetLarge,"VGG16":VGG16}
#"InceptionV3":InceptionV3,"DenseNet121":DenseNet121,
 #       "DenseNet169":DenseNet169,"DenseNet201":DenseNet201,"Xception":Xception, 
 #       "InceptionResNetV2":InceptionResNetV2,
#"ResNet50":ResNet50, 
#"VGG16":VGG16,"VGG16":VGG19,"NASNetMobile":NASNetMobile

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

classes = ['coat_length_labels','collar_design_labels', 'lapel_design_labels',
           'neck_design_labels','neckline_design_labels',
           'pant_length_labels', 'skirt_length_labels', 
           'sleeve_length_labels']    
fai_result = []
label_count = {'coat_length_labels':8,
               'collar_design_labels':5, 
               'lapel_design_labels':5,
               'neck_design_labels':5,
               'neckline_design_labels':10,
               'pant_length_labels':6, 
               'skirt_length_labels':6, 
               'sleeve_length_labels':9}
#####################
width = 299
#####################
#X = {}
#y = {}
#df_train = {}
#df_load = {}
#for cur_class in classes:
   # df_train[cur_class] = pd.read_csv('../train/Annotations/{0}.csv'.format(cur_class), header=None)
   # X[cur_class] = np.zeros((len(df_train[cur_class]), width, width, 3), dtype=np.uint8)
    #y[cur_class] = np.zeros((len(df_train[cur_class]), label_count[cur_class]), dtype=np.uint8)
    
    #df_train[cur_class].columns = ['image_id', 'class', 'label']
    #df_load[cur_class] = df_train[cur_class].copy()
    #df_load[cur_class].reset_index(inplace=True)
    #del df_load[cur_class]['index']
    #样本总数 
    #print("各类样本总数{0}:{1}".format(cur_class,len(df_load[cur_class])))
    
df = pd.read_csv('../train/Annotations/train.csv', header=None)
df.columns = ['image_id', 'class', 'label']
dfl = df.copy()
dfl.reset_index(inplace=True)
del dfl['index']
nn = len(dfl)
XX = np.zeros((nn, width, width, 3), dtype=np.uint8)
    
#yyy = np.zeros((nn, 8, 5, 5, 5, 10, 6, 6, 9), dtype=np.uint8)
    
yy = [np.zeros((nn, label_count[x])) for x in label_count.keys()]
#print(yy)
print("载入图像到内存..") 
for i in tqdm(range(nn)):
        
        
    #n_class = len(df_load['label'][0])
    #if len(tmp_label) == label_count[df_load['class'][i]]:
    XX[i] = cv2.resize(cv2.imread('../train/{0}'.format(dfl['image_id'][i])), (width, width))
print("图像已载入内存,开始载入转换的标签..")    
for i in tqdm(range(nn)):
        
    tmp_label = dfl['label'][i]
    if dfl['class'][i] == 'coat_length_labels':
        yy[0][i][tmp_label.find('y')] = 1
            
    elif dfl['class'][i] == 'collar_design_labels':
        yy[1][i][tmp_label.find('y')] = 1
            
    elif dfl['class'][i] == 'lapel_design_labels':
        yy[2][i][tmp_label.find('y')] = 1
            
    elif dfl['class'][i] == 'neck_design_labels':
        yy[3][i][tmp_label.find('y')] = 1
            
    elif dfl['class'][i] == 'neckline_design_labels':
        yy[4][i][tmp_label.find('y')] = 1
            
    elif dfl['class'][i] == 'pant_length_labels':
        yy[5][i][tmp_label.find('y')] = 1
            
    elif dfl['class'][i] == 'skirt_length_labels':
        yy[6][i][tmp_label.find('y')] = 1
            
    else:
        yy[7][i][tmp_label.find('y')] = 1
          
X_train, X_valid, y_train, y_valid = train_test_split(XX, 
        yy, test_size=0.1, random_state=50)



Using TensorFlow backend.
  0%|          | 13/89683 [00:00<11:34, 129.21it/s]

载入图像到内存..


  2%|▏         | 1619/89683 [00:00<00:05, 16183.30it/s]

图像已载入内存,开始载入转换的标签..


100%|██████████| 89683/89683 [00:13<00:00, 6738.83it/s] 


ValueError: Found input variables with inconsistent numbers of samples: [89683, 8]

In [ ]:
for KEY, MODLE in MODELS.items():
    
    
    #为299*299,设置如下
    ppreprocess = preprocess_input
    if KEY in ["InceptionV3","Xception", "InceptionResNetV2"]:
        width = 299
    elif KEY == "NASNetLarge":
        width = 331
    else:
        width = 224
        ppreprocess = imagenet_utils.preprocess_input 
    print('######################在{0}下训练8个分类器####################'.format(KEY))
    cnn_model = MODLE(include_top=False, input_shape=(width, width, 3), weights='imagenet',pooling='avg')
    inputs = Input((width, width, 3))
    x = inputs
    x = Lambda(ppreprocess, name='preprocessing')(x)
    x = cnn_model(x)
        #下面是新加的层
        #x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
        #x = Flatten(name='flatten')(x)
        #x = Dense(2048, activation='relu', name='fc1')(x)
        # n_class为对应属性的分类个数
        #x = Dense(256, activation='relu', kernel_initializer=initializers.he_uniform(seed=None),name='fc1')(x)
        #x = Dense(n_class, activation='softmax', name='softmax')(x)
    #x = Dense(512, activation='relu', kernel_initializer=initializers.he_uniform(seed=None),name='fc2')(x)
    #x = Dropout(0.5)(x)
    x = [Dense(count, activation='softmax', name=name)(x) for name, count in label_count.items()]
    model = Model(inputs, x)
    adam = Adam(lr=0.001)

        #多GPU训练,因为keras设计的自动保存最好模型,但是多GPU训练,其save()就没法用了
        #model = multi_gpu_model(model, 2)  

    model.compile(optimizer=adam,
                    loss='categorical_crossentropy',
                   metrics=['accuracy'])
    print("模型初始化完毕") 
    

        #sgd = SGD(lr=learning_rate, decay=learning_rate/nb_epoch, momentum=0.9, nesterov=True)
        #adam = optimizers.Adam(lr=1e-4)
        #optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True))
       
    # Callback that implements learning rate schedule
    #schedule = Step([20], [1e-4, 1e-6])
    #history = model.fit(X_train, Y_train,
    #                    batch_size=batch_size, nb_epoch=nb_epoch, validation_data=(X_test,Y_test),
    #                    callbacks=[
    #                           schedule,
    #                           keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0,save_best_only=True, mode='auto')

    # 该回调函数将在每个epoch后保存模型到filepath
    #keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0,save_best_only=True, mode='auto')
    # 当监测值不再改善时，该回调函数将中止训练.
    #当early stop被激活（如发现loss相比上一个epoch训练没有下降），则经过patience个epoch后停止训练
    #keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')

        #设置训练完之后,最好的模型保存路径
        #checkpointer = ModelCheckpoint(filepath='../models/{0}.best.h5'.format(KEY), verbose=1, 
                #                        save_best_only=True)
        #训练开始,并保存训练过程的loss和acc变化
    h = model.fit(X_train, y_train, batch_size=8, epochs=32, 
                      #callbacks=[EarlyStopping(patience=12), checkpointer], 
                      shuffle=True, 
                      validation_data=(X_valid,y_valid))
    print("模型训练完毕!")
    #保存模型
    print("开始保存模型")
    model.save_weights('../model.h5')
    print("下面开始测试")
    df_test = pd.read_csv('../test/Tests/question.csv', header=None)
    df_test.columns = ['image_id', 'class', 'x']
    del df_test['x']  
    df_load = df_test.copy()
    df_load.reset_index(inplace=True)
    del df_load['index']  
    nn = len(df_load)
    X_test = np.zeros((nn, width, width, 3), dtype=np.uint8)
    print("载入测试集图像到内存")
    for i in tqdm(range(nn)):
        X_test[i] = cv2.resize(cv2.imread('../test/{0}'.format(df_load['image_id'][i])), (width, width))
        #测试集上预测并输出结果
    test_np = model.predict(X_test, batch_size=128)
    print(test_np)
    
    

In [ ]:
#reasult = [np.zeros((nn, label_count[x])) for x in label_count.keys()]
result = []
for i in tqdm(range(nn)):
        
    #tmp_label = df_load['label'][i]
    #n_class = len(df_load['label'][0])
    #if len(tmp_label) == label_count[df_load['class'][i]]:
    tmp_result = ''
    if df_load['class'][i] == 'coat_length_labels':
        tmp_listdd = test_np[0][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])
            
    elif df_load['class'][i] == 'collar_design_labels':
        tmp_listdd = test_np[1][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])
        
    elif df_load['class'][i] == 'lapel_design_labels':
        tmp_listdd = test_np[2][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])
        
    elif df_load['class'][i] == 'neck_design_labels':
        tmp_listdd = test_np[3][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])
        
            
    elif df_load['class'][i] == 'neckline_design_labels':
        tmp_listdd = test_np[4][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])
        
    elif df_load['class'][i] == 'pant_length_labels':
        tmp_listdd = test_np[5][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])
        
    elif df_load['class'][i] == 'skirt_length_labels':
        tmp_listdd = test_np[6][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])
        
    else:
        tmp_listdd = test_np[7][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])

df_load['result'] = result     
df_load.to_csv('../result/{0}/{0}.csv'.format(KEY), header=None, index=False)




In [ ]:
print("打印原预测列表:")
print(test_np)